- Baseline: Grid pequeño 100x100
- TODO resetear modelo y graficas (score y mean score)
- modificar: max_memory, batch_size, learning_rate
- Transfer learning
- Añadir reward para acercarse al premio (grid más grande)

In [6]:
!pip install pygame

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 25.4 MB/s eta 0:00:0000:0100:01


In [2]:
!python snake_game_human.py --block_size 20 --speed 10 --width 800 --height 600

Traceback (most recent call last):
  File "/Users/carlosperezricardo/Documents/GitHub/Reinforcement-Learning-Snake/snake_game_human.py", line 1, in <module>
    import pygame
ModuleNotFoundError: No module named 'pygame'


#AI GAME

- Init reward: 0
- Reward per collision: -10
- Reward per divagar / no anar a pel premi: -20
    - Ambdos casos son GAME OVER
- Reward per menjar: +10
- Food 

# AGENT

## STATES

Booleans que indiquen si:

1. Danger straight
2. Danger right
3. Danger left
4. dir left
5. dir right
6. dir up
7. dir down
8. el menjar està a l'esquerra: game.food.x < game.head.x,  # food left
9. el menjar està a la dreta: game.food.x > game.head.x,  # food right
10. el menjar està a sota: game.food.y < game.head.y,  # food up
11. el menjar està a sobre: game.food.y > game.head.y  # food down


State: torch.Size([11])
Next state: torch.Size([11])
Action: torch.Size([3])
Reward: torch.Size([])
Prediction: torch.Size([1, 3])

In [2]:
!python agent.py --block_size 20 --speed 30 --width 400 --height 400 --discount 0.9 --max_memory 100_000 --batch_size 1000 --learning_rate 0.001 --max_n_games 1_000

pygame 2.5.2 (SDL 2.28.3, Python 3.12.2)
Hello from the pygame community. https://www.pygame.org/contribute.html
/Users/carlosperezricardo/Documents/GitHub/Reinforcement-Learning-Snake/model.py:36: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:281.)
  state = torch.tensor(state, dtype=torch.float)
Game 1 Score 0 Record: 0
Figure(640x480)
Figure(640x480)
Game 2 Score 0 Record: 0
Figure(640x480)
Figure(640x480)
Game 3 Score 0 Record: 0
Figure(640x480)
Figure(640x480)
Game 4 Score 0 Record: 0
Figure(640x480)
Figure(640x480)
Game 5 Score 1 Record: 1
Figure(640x480)
Figure(640x480)
Game 6 Score 0 Record: 1
Figure(640x480)
Figure(640x480)
Game 7 Score 0 Record: 1
Figure(640x480)
Figure(640x480)
Game 8 Score 0 Record: 1
Figure(640x480)
Figure(640x480)